## CTCB Practical 3: Molecular Dynamics

In this practical, you will prepare and run molecular dynamics (MD) simulations for the cannabinoid 2 (CB2) receptor-ligand complex that you also looked at in Practical 1. MD simulations are a powerful computational tool used to study the behavior of (macro)molecules at the atomic level. Here, you will go through the steps required to prepare and run MD simulations using the PyMemDyn Python Package. Normally, these steps are all run at once, but here they are run individually to better understand what each step does and why they are needed for a successful MD simulation on a membrane protein.

We recommend you to read up on PyMemDyn and related webserver in the following papers: \
https://portlandpress.com/biochemsoctrans/article-abstract/41/1/205/68208/Characterization-of-the-dynamic-events-of-GPCRs-by \
https://academic.oup.com/nar/article/44/W1/W455/2499371

Below, you will find the relevant code blocks that you can execute by using the play button on top, or by typing Shift+Enter. You will use PyMOL to visualize the process of creating an MD simulation, and ultimately the MD simulation itself.

If you have any questions about what the code is doing, please ask one of the TAs!

### Initiate PyMemDyn

In [ ]:
# Import modules
import os
import sys
import pandas as pd

# Import GROMACS function
from gromacs.fileformats.xvg import XVG

# Import PyMemDyn module/function
sys.path.insert(0, '../')
import settings
from pymemdyn import Run

In [ ]:
if not os.getcwd().endswith('rundir'):
    os.chdir('rundir')

### Input

Our input consists out of two parts, our protein and our ligand:

**Protein**: \
The cryo-EM structure of a GPCR-G Protein complex (6KPF; https://www.rcsb.org/structure/6KPF) is used as the starting point for the protein model. For our simulations we are only interested in CB2 (colored in Magenta), thus we firstly removed the G protein from the complex. Secondly, we looked for gaps that result in loose ends within the sequence of the original structure. Gaps mainly are present in flexible regions of the proteins which cannot be resolved through crystallography and imaging. In our structure we see a gap in the ICL3 between TM5 and TM6. We connected the loose ends to increase the stability of our protein during the MD simulations.


In [ ]:
# copy protein input files to working directory
!cp ../data/protein.pdb .

 - Open the protein.pdb file in PyMOL so you can see the processed structure and compare it with the original structure on the PDB.

**Ligand**: \
We have docked our ligand onto the protein such as you have done in previous Practicals. For our ligand, we have pre-calculated the topology and OPLS-AA force field parameters (.itp and .ff) for E3R using the LigParGen webserver (http://zarbi.chem.yale.edu/ligpargen/). This is an open-source tool that allows for the calculation of the force field parameters of organic ligands.

""" \
*Force fields*: \
In MD simulations, the motions of atoms and molecules are simulated by numerically solving Newton's equations of motion, which require information about the forces acting on the particles. 

To accurately model the behavior of molecules in MD simulations, it is necessary to use force field parameters. A force field is a mathematical function that describes the potential energy of a molecule as a function of its position and orientation in space. The force field parameters are the numerical constants in this function that determine the strength of the various types of interactions between atoms or molecules. 

Force field parameters include bond lengths, bond angles, dihedral angles, van der Waals interactions, and electrostatic interactions. \
"""

""" \
*Topology files*: \
Topology files (.itp or .top) are used to connect the atoms found within the structure files (.pdb or .gro) and correctly link them with properties found within other files, such as force fields and positional restraints. \
"""

In [ ]:
# copy ligand input files to working directory
!cp ../data/E3R.pdb .
!cp ../data/E3R.itp .
!cp ../data/E3R.ff .


### Running PyMemDyn

With your input gathered, you are now ready to execute PyMemDyn step-by-step. 

First, we instantiate the Run class of PyMemDyn. This is a top level object that takes run time variables and stores them in an object so we can use them in the next steps. In this step we communicate with PyMemDyn what our protein and ligand files are, and which restraints we wish to use during MD. In this case we use Venkatakrishnan/Ballesteros-Weinstein (bw) positional restraints. You can read more about these restaints on https://academic.oup.com/nar/article/44/W1/W455/2499371.

""" \
*Restraints*: \
Restraints are commonly used in molecular dynamics (MD) simulations to constrain certain degrees of freedom of the system being simulated. 

We use restraints to maintain the structural integrity of the system. Restraints can be used to maintain the structural integrity of a molecule or system, by preventing bond lengths, bond angles, or dihedral angles from deviating too far from their ideal values.

There are also other applications for restrains such as sampling conformational space, improve convergence and study specific interactions. If you are interested in these applications, we encourage you to investigate them yourselves. \
"""

In [ ]:
run = Run(own_dir = os.getcwd(),
          repo_dir = settings.TEMPLATES_DIR,
          pdb = 'protein.pdb',
          ligand = 'E3R',
          alosteric = None,
          waters = None,
          ions = None,
          cho = None,
          ligpargen = None,
          library = None,
          restraint = 'bw',
          queue = "",
          debug = True
)

PyMemDyn consists of many individual steps. The command below returns a list of each of the steps run by the PyMemDyn module. We will use the keywords it outputs in the following command blocks to run the individual steps in the program.
Note that PyMemDyn normally runs everything in one go (this was the intention of the program), but for teaching purposes it makes more sense to carefully look at each of the individual steps.

In [ ]:
run.moldyn_notebook_info()

#### Setup protein-ligand complex

PyMemDyn communicates with the MD simulation software GROMACS. GROMACS however requires three types of files to be able to simulate the MD of an object: the structural file (.pdb or .gro), the topology file (.itp or .top) and the force fields (.itp or .ff). For our ligand we have pre-calculated these using LigParGen, but for our protein we use GROMACS to calculate this.

**pdb2gmx** converts the protein.pdb file into three files:
- *proteinopls.pdb* contains the structural file of the protein
- *protein.top* contains the topology of the protein
- *posre.itp* contains the positional restraints (by default with no restraints)


**set_itp** then makes a backup of the topology file for later use.

In [ ]:
run.moldyn_notebook_run("pdb2gmx")

In [ ]:
run.moldyn_notebook_run("set_itp")

With the topology and restraints files generated for the protein, we want to combine the structural files of the protein and the ligand. **concat** merges the protein and ligand structure files. This provides us with *tmp_proteinopls.pdb*. 

In [ ]:
run.moldyn_notebook_run("concat")

### Setup initial simulation box

For both processing our structures and running the MD simulations, we need to define the box in which our system is present. 

**editconf** calculates the inital box which contains our protein and ligand. The **editconf** output lets us investigate the centres, vectors and angles of the system in addition to the total volume. We start by calculating a box with 2.0 Å distance around the complex which provides us with a loose-fitting box around the complex. \
**set_protein_size** then saves the x and y dimensions of the box for later use. We calculate this on a loose-fitting box since we still intend to insert the protein within a membrane \
**editconf2** reduces the box to 0.8 Å  around the complex, which gives us a much tighter box. \
**set_protein_size2** then saves the z dimension of the box for later use. We reduced the size since we need less distance correctly simulate waters above and below the system compared to the membrane.


""" \
*simulation box*: \
In molecular dynamics (MD) simulations, a simulation box is defined to contain the molecular system being simulated. The simulation box is a three-dimensional space that includes all of the atoms or molecules in the system. There are several reasons why a simulation box is necessary for MD simulations:

To define periodic boundary conditions: In MD simulations, periodic boundary conditions are often used to simulate an infinite system by replicating the simulation box in all three dimensions. This can improve the accuracy of the simulation by reducing the influence of the system size on the simulation results.

To avoid edge effects: By defining a simulation box that is sufficiently large, edge effects can be minimized. Edge effects occur when atoms or molecules at the boundary of the simulation box interact differently than atoms or molecules in the interior of the box. This can lead to unphysical behavior and inaccuracies in the simulation results.

To define the density of the system: The size of the simulation box determines the density of the system being simulated. By adjusting the size of the simulation box, the density of the system can be controlled, which can affect the dynamics and properties of the system.

To control the volume of the system: The size of the simulation box can also be used to control the volume of the system being simulated. By constraining the volume of the simulation box, pressure and other thermodynamic properties can be controlled. \
"""

In [ ]:
run.moldyn_notebook_run("editconf")

In [ ]:
run.moldyn_notebook_run("set_protein_size")

In [ ]:
run.moldyn_notebook_run("editconf2")

In [ ]:
run.moldyn_notebook_run("set_protein_size2")

Now the protein-ligand complex is assembled and and our initial box is defined. \
- Open the proteinopls.pdb file in PyMOL to see what the complex currently looks like.

#### Prepare membrane embedding

We currently have the protein-ligand complex ready, but it is still floating within a void. CB2 is a membrane bound protein, so for our MD simulation, we will have to setup the system to represent this environment. Firstly, we will embed our protein within a POPC membrane model. 

**set_popc** retrieves our membrane model we will use later. 
- After running the command, look at popc.pdb with PyMOL to see how the model initially looks like. 

**editconf3** changes the box dimensions of *proteinopls.pdb* to prepare for membrane embedding. In addition, we change the shape to triclinic as a precursor to our final shape. 

In [ ]:
run.moldyn_notebook_run("set_popc")

In [ ]:
run.moldyn_notebook_run("editconf3")

#### Prepare ligand parameters

For our ligand, we had the structure, topology and force field files pre-calculated as input. We however still need to create an index and generate the positional restraint for it. \
**make_ndx_lig**: creates a GROMACS index file to create index group for the ligand. It helps keep track of atoms during pre-processing and MD simulation\
**genrestr_lig**: creates positional restraint file (posre_lig.itp) similar to the positional restraint file generated for the protein in **pdb2gmx**.


In [ ]:
run.moldyn_notebook_run("make_ndx_lig")

In [ ]:
run.moldyn_notebook_run("genrestr_lig")

#### Membrane embedding

**editconf4** create a simulation box for popc.pdb. The dimensions of the box were determined by the **set_protein_size** and **set_protein_size2** commands. Note that now the new box centre for the z-axis (3rd colunm) is the same as for proteinopls.pdb (**editconf3**). This ensures that the membrane will be inserted at the correct height of the protein. \
**make_topol** makes a topology file (topol.top) linking the topopology and positional restraint files of all objects. Currently in the bottom of the file, you can see there is 1 protein and 1 ligand present. Later steps will add more molecules when they are incorporated within the system. \
**editconf5** is a dummy step only present to check if the previous steps have executed correctly.

In [ ]:
run.moldyn_notebook_run("editconf4")

In [ ]:
run.moldyn_notebook_run("make_topol")

In [ ]:
run.moldyn_notebook_run("editconf5")

We now have both the protein-ligand complex (proteinopls.pdb) and the lipid bilayer (popc.gro) prepared for embedding. We use **solvate** to surround the protein with lipids. In essence, both structures are overlayed, and lipids that overlap with either the protein or the ligand are removed. 

In [ ]:
run.moldyn_notebook_run("solvate")

Now we can check whether embedding has gone correctly.

- Firstly, open *topol.top*. Here you should find POP to be added to the bottom of the file. This indicates the number of lipid molecules that were added to the system.

- Secondly, open *protpopc.pdb* using PyMOL. You can see that the protein is now surrounded by bilayer of POPC lipids on three sides. This means embedding a successfull embedding. Due to the infinite box principle (see """*simulation box*""") in practice all sides of the protein surrounded by lipids. 

#### Solvate Protein-Membrane System with Waters

You should now have your protein correctly embedded within the membrane. This however still is not a complete system. The intra- and extracellular parts of the system still remain voids. In the following steps we will prepare our system again for solvation using a water model. We do this in a similar fashion as we have done for the membrane before:

**set_water** retrieves our membrane model we will use later.

- After running the command, look at water.pdb with PyMOL to see how the water model initially looks like. 


water.pdb is retrieved from an equilibrated membrane-water system. Noteworth is that there are no waters where we will insert our membrane-protein system. This reduces the chance of accidently inserting waters within the membrane, thus destabilizing the simulation.

In [ ]:
run.moldyn_notebook_run("set_water")

**editconf6** and **editconf7** prepare the simulation boxes for both water.pdb and protpopc.pdb. Here again, we ensure the z-axic centre and vectors are equal before solvation. 

In [ ]:
run.moldyn_notebook_run("editconf6")

In [ ]:
run.moldyn_notebook_run("editconf7")

We now have both the embedded membrane (protpopc.pdb) and the water model (water.pdb) prepared for solvation. We use **solvate2** to solvate the system with waters. In essence, both structures are overlayed, and waters that overlap with the embedded membrane are removed. 

In [ ]:
run.moldyn_notebook_run("solvate2")

Now we can check whether embedding has gone correctly.

- Firstly, open *topol.top*. Here you should find SOL to be added to the bottom of the file. This indicates the number of water molecules that were added to the system.

- Secondly, open *tmp.pdb* using PyMOL. You can see that the membrane is now surrounded intra- and extracellular with water molecules. Note: the alignment between the water model and the membrane system lookes off, but in reality it is embedded correctly. (See: infinite box principle ("""*simulation box*""")). This means embedding a successfull embedding.

With our system now solvated in water, we will continue with preparing all files required for MD simulation. Firstly, to keep track of the the intra- and extracellular lipids, we will remake the topology file:

**count_lipids**: Count the number of intra- and extracellular lipids within the membrane. \
**make_topol**: Reset the current topology file. \
**make_topol_lipids**: Make a topology file which differentiates between intra- and extracellular lipids.

In [ ]:
run.moldyn_notebook_run("count_lipids")

In [ ]:
run.moldyn_notebook_run("make_topol2")

In [ ]:
run.moldyn_notebook_run("make_topol_lipids")

#### Prepare MD simulation (1)

In the following steps we retrieve all MD parameter files, structure, topology and index files to create a singular MD input file. In addition, we prepare the MD simulation by adjusting the box size and shape, and by making the global charge within the system neutral.

**make_ffoplsaanb** creates a singular force field file (ffoplsaanb_mod.itp) containing all forcefields present within the system. The files start with our ligand forcefield parametes (as generated by LigParGen) and is followed by a dictionary containing all common forcefields. In this dictionary you can find the lipid, water and ion forcefields as well. 


In [ ]:
run.moldyn_notebook_run("make_ffoplsaanb")

**set_grommp** retrieves pre-made template files which are required for MD simulation. These include the topology files for the atoms we added/will add to our system (popc.itp (lipids), spc.itp (waters), and ions.itp (ions)). In addition other files parameter files are gathered: steep.mdp (MD parameters), ffoplsaanbon_mod.itp and ffoplsaa_mod.itp (additional dictionaries for atom/bond parameters).

- Have a look at spc.itp and find the corresponding forcefields in ffoplsaanb_mod. (Tip: use Ctr-F)
- Also have a look at steep.mdp to find the global parameters of our MD simulation.

In [ ]:
run.moldyn_notebook_run("set_grompp")

**set_chains** redoes atom numbering for correct atom indexing when dealing with multimeric protein structures. In our case we are simulating a monomer, so this step will not do anything.

In [ ]:
run.moldyn_notebook_run("set_chains")

**make_ndx**: Make a GROMACS index file to keep track of which atom is what during the simulation. 
 - Open *index.ndx* to see what this file looks like. If you search for "[" you can easily loop through the different categories.


In [ ]:
run.moldyn_notebook_run("make_ndx")

**grompp** is a GROMACS funtion that combines the structural file (tmp.pdb), MD parameters (steep.mdp), topology file containing forcefields and restraints (topol.top), and index file (index.ndx) to create a singular MD input file (topol.tpr) for running MD simulations. In addition, it also gives us information about the global system. The first instances of **grompp** we will run will be to retrieve the latter to make our last modifications to the system.

In [ ]:
run.moldyn_notebook_run("grompp")

**trjconv** re-adjusts tmp.pdb based on tmp.pdb and topol.tpr (our newly generated global MD file).

- Open tmp.pdb. As you can see the water layer is now correctly placed above the membrane. This is done by changing the perodicity repsentation of the system. No other changes are done in this step. 

Note: The atom identifiers in this structure file are not labeling all atoms correctly. This has no influence on the further steps and will be resolved at the end.

In [ ]:
run.moldyn_notebook_run("trjconv")

#### Add Ions to System


With our system now gobally defined, we can look at its charge. For a stable MD simulation, we want the charge to be neutral. To achieve this, we first find the global charge using **get_charge** and then replacing water atoms with ions in our model with **genion**.

In [ ]:
run.moldyn_notebook_run("get_charge")

In [ ]:
run.moldyn_notebook_run("genion")

The system has now been made neutral by replacing water atoms with monoatomic ions. In our case we had a positive charge, so Cl- was added. In the case of a negative charge, Na+ would have been added. 

 - We will wait with opening output.pdb until after **trjconf2**, as the structure file first has to be re-adjusted.

 With **grompp2** and **trjconv2** we remake the MD input file (topol.tpr) and use it to re-adjust the simulation box with the ions included.

In [ ]:
run.moldyn_notebook_run("grompp2")

In [ ]:
run.moldyn_notebook_run("trjconv2")

- Have a look at output.pdb to see our current system with ions added.

#### Create final MD input

We now run **grompp3**, **trjconv3**, and **clean_pdb** to retrieve our final input files for MD simulation.

**Grommp3**: Redo creating MD input file  \
**Trajconv3**: Redo the periodicity representation to represent a hexagon. This shape allows for the smallest box size for a system while still retaining an infinite system. I.e.: a sqare retains periodic boundry conditions, but the vertices contain unneccesary data. In contrast, a circle contains as little information around the protein as possible, but looses the periodic boundary condition. Placing multiple circles next to eachother will always result in "empty" holes, while this does not happen with a hexagon or square. \
**clean_pdb** corrects atom labeling to make PDB visualization nicer :)

In [ ]:
run.moldyn_notebook_run("grompp3")

In [ ]:
run.moldyn_notebook_run("trjconv3")

In [ ]:
run.moldyn_notebook_run("clean_pdb")

The final system for MD simulation has now been created. The system has now been converted into an hexagonal shape. \
- Open the hexagon.pdb file in PyMOL to see what the system now looks like.

### MD Simulation

**After** you have completed all steps above, you would normally start running the MD simulation within PyMemDyn. However it takes a few hours to complete on a pretty large cluster,
so we have already done it for you.

The output from a PyMemDyn run can be found in the finalOutput folder, which we copy in here now.

In [ ]:
!cp -r ../data/finalOutput .

The table below shows steps of the MD simulation. Note that over the time, we reduce the number of restaints and their force constraint. Also keep the time for each step in mind for the next analysis steps.

|   STAGE    | RESTRAINED ATOMS        | FORCE CONSTANT       | TIME           |
|:----------:|:-----------------------:|:--------------------:|:--------------:|
|  -         |   -                     |kJ/(mol�nm^2)        | ns             |
|Minimization|   -                     | -                    |(Max. 500 steps)|
|Equil. 1    |Protein Heavy Atoms      | 1000                 | 0.5            |
|Equil. 2    |Protein Heavy Atoms      | 800                  | 0.5            |
|Equil. 3    |Protein Heavy Atoms      | 600                  | 0.5            |
|Equil. 4    |Protein Heavy Atoms      | 400                  | 0.5            |
|Equil. 5    |Protein Heavy Atoms      | 200                  | 0.5            |
|Equil. 6    |Venkatakrishnan Pairs /  | 200 /                | 2.5 /          |
|            |C-alpha Atoms            | 200                  | 2.5            |



#### MD reports

Typically, it's useful to analyze a few properties, like temperature and pressure, but also RMSD and RMSF (if you don't know what those are please Google it or ask a TA) are good indicators for the stability of your simulation. We can use basic plotting to work with GROMACS output files, e.g.: to check the temperature during your simulation run:

In [ ]:
#Define a helper function that plots .xvg files:
def plot_xvg(xvgfile1, xvgfile2=None): 
    df = XVG(xvgfile1).to_df()

    if xvgfile2 != None:
        df2 = XVG(xvgfile2).to_df()
        df2 = df2.drop(0)
        df2.iloc[:,0] = df2.iloc[:,0] + 2500
        df = pd.concat([df, df2], axis=0)

    plot = df.plot(x=df.columns[0], ylabel=df.columns[1], legend=None, title=xvgfile1)    

    return plot

We start by investigating the base properties of our system (temperature, pressure, energy and volume):

In [ ]:
plot_xvg('finalOutput/reports/temp.xvg', 'finalOutput/reports/temp2.xvg')

In [ ]:
plot_xvg('finalOutput/reports/pressure.xvg', 'finalOutput/reports/pressure2.xvg')

In [ ]:
plot_xvg('finalOutput/reports/tot_ener.xvg', 'finalOutput/reports/tot_ener2.xvg')

In [ ]:
plot_xvg('finalOutput/reports/volume.xvg', 'finalOutput/reports/volume2.xvg')

- Write a short description for each of the figures. Discuss what you are seeing with your fellow students and/or TAs. 

We now do the same for the RMSD and RMSF figures:

In [ ]:
plot_xvg('finalOutput/reports/rmsd-calpha-vs-start.xvg')

In [ ]:
plot_xvg('finalOutput/reports/rmsf-per-residue.xvg')

- Write a short description for each of the figures. Discuss what you are seeing with your fellow students and/or TAs. 

#### MD trajectory

Next, we can have a look at the trajectory. Open PyMOL and change to the finalOutput directory. There, you can load a PyMOL script (.pml) generated by PyMemDyn, by typing @load_gpcr.pml

- Can you see when the restraints are released in the simulation? 
- Can you link that to events described in the PyMemDyn and GPCR-ModSim papers?
- Do you think the ligand looks stable?
- What could you do to further assess ligand stability?
- Do we know anything about the binding affinity now?

Thanks for participating in this last computerlab, as always please ask your TA any questions if something is unclear!